### 概要
1. 动态规划流程：
    - 暴力递归
    - 带备忘录的递归
    - 迭代的动态规划
2. 思考步骤：
    - 找到状态，选择和base case
    - 明确dp数组和函数的定义
    - 寻找状态间的联系，明确状态转移方程
3. 其他心得
    - 动态规划的一般问题形式是求最值
    - 动态规划问题的核心是穷举
    - 通过子问题的最值得到原问题的最值
    - 暴力穷举，存在重叠子问题
    - 动态规划问题一定具备最优子结构
    - 只有列出正确的状态转移方程，才能正确的穷举
    - 最优子问题，子问题必须相互独立
4. 算法实战
    - 4.1 股票问题
    - 4.2 背包问题
    - 4.3 打家劫舍问题
    - 4.4 零钱兑换问题
    - 4.5 最小编辑距离
    - 4.6 鸡蛋掉落问题
    - 4.7 爬楼梯问题
    - 4.8 博弈问题
    - 4.9 最小重叠子区间
    
5. 本文主要参阅：
    - 大神labuladong：https://labuladong.gitbook.io/algo/dong-tai-gui-hua-xi-lie/
    - 力扣赛题：https://leetcode-cn.com/problems/

### 股票问题-买卖股票的最佳时机（121，122，123，188）

- 题目：  
给定一个数组，它的第 i 个元素是一支给定股票第 i 天的价格。  
如果你最多只允许完成一笔交易（即买入和卖出一支股票），设计一个算法来计算你所能获取的最大利润。  
注意你不能在买入股票前卖出股票。  
示例 1:  
输入: [7,1,5,3,6,4]  
输出: 5  
解释: 在第 2 天（股票价格 = 1）的时候买入，在第 5 天（股票价格 = 6）的时候卖出，最大利润 = 6-1 = 5 。  
     注意利润不能是 7-1 = 6, 因为卖出价格需要大于买入价格。  
示例 2:  
输入: [7,6,4,3,1]  
输出: 0  
解释: 在这种情况下, 没有交易完成, 所以最大利润为 0。  

- 题目目标：
     - 1 最多买一次，卖一次，注意，可以不买，但一定会卖，既然买了，卖掉的话，多少会回点本
     - 2 分别在哪一天交易，利润最大

- 解题思路：
     - 1 状态：
          - 天数
          - 交易次数
          - 是否持有股票
     - 2 选择：
          - 买或者不买，卖或者不卖，不买和不卖可算作同一种类型的选择，因为对状态无影响
          - 即选择为：无操作，买入，卖出
     - 3 base case:
          - 到了交易的第一天或者交易次数用完
     - 4 dp函数定义：
          - dp[i][k][j] = p
          - i表示交易的第i天
          - k表示第i天最大可交易次数
              - k-1表示第i-1天的最大可交易次数，因为i-1=>i，i的最大可交易次数>=k-1
              - 因为最后一定会卖出，所以定义卖出一次算交易一次
          - j表示：1=持有股票，0=没有股票
          - p表示第i天进行k次交易其持有/没有股票时的最大利润
     - 5 状态转移方程：
          - dp[i][k][0]=max(dp[i-1][k][0]+0,dp[i-1][k-1][1]+prices[i])
          - 含义：第i天没有股票=max(第i-1天进行了k次交易且没有股票+不交易，第i-1天持有股票+卖掉)
          - dp[i][k][1]=max(dp[i-1][k][1]+0,dp[i-1][k][0]-prices[i])
          - 含义：第i天持有股票=max(第i-1天进行了k次交易且持有股票+不交易，第i-1天进行了k-1次交易且没有股票+买入)
          - base case:
              - dp[-1][k][0] = 0
              - 含义：第-1天可交易k次且没有股票，利润肯定为0
              - dp[-1][k][1] = -inf
              - 含义：第-1天可交易k次且持有股票，不可能，利润肯定为负无穷
              - dp[-1][0][0] = 0
              - 含义：第-1天可交易0次且没有股票，利润肯定为0
              - dp[-1][0][1] = -inf
              - 含义：第-1天可交易k次且持有股票，不可能，利润肯定为负无穷


In [67]:
class Solution:
    
    def maxProfit(self, prices: List[int]) -> int:
        if not prices:return 0
        return self.solve_k(prices,float('inf'))
    
    #允许k次交易
    def solve_k(self,prices,k):
        n = len(prices)
        if k >= (n//2):
            return self.solve_inf(prices)
        dp = [[[0,0] for i in range(k+1)] for i in range(n)]
        for i in range(n):
            for j in range(k,0,-1):
                if i == 0:
                    #第-1天最大可以交易次数为0
#                     dp[i][k][0] = max(dp[-1][0][0]+0,dp[-1][0][1]+prices[i])
#                     dp[i][k][1] = max(dp[-1][0][1]+0,dp[-1][0][0]-prices[i])
                    dp[i][j][0] = max(0+0,float('-inf')+prices[i])
                    dp[i][j][1] = max(float('-inf')+0,0-prices[i])
                    continue
                dp[i][j][0] = max(dp[i-1][j][0]+0,dp[i-1][j][1]+prices[i])
                dp[i][j][1] = max(dp[i-1][j][1]+0,dp[i-1][j-1][0]-prices[i])
        return dp[n-1][k][0]
    
    #交易次数没有限制
    def solve_inf(self,prices):
        n = len(prices)
        dp = [[0,0] for i in range(n)]
        for i in range(n):
            if i == 0:
                #第-1天最大可以交易次数为0
#                 dp[i][0] = max(dp[-1][0]+0,dp[-1][1]+prices[i])
#                 dp[i][1] = max(dp[-1][1]+0,dp[-1][0]-prices[i])
                dp[i][0] = max(0+0,float('-inf')+prices[i])
                dp[i][1] = max(float('-inf')+0,0-prices[i])
                continue
            dp[i][0] = max(dp[i-1][0]+0,dp[i-1][1]+prices[i])
            dp[i][1] = max(dp[i-1][1]+0,dp[i-1][0]-prices[i])
                
        return dp[n-1][0]
    
    #交易次数没有限制，但是有冷冻费
    def solve_inf_colddown(self,prices):
        n = len(prices)
        dp = [[0,0] for i in range(n)]
        for i in range(n):
            if i == 0:
                #第-1天最大可以交易次数为0
#                 dp[i][0] = max(dp[-1][0]+0,dp[-1][1]+prices[i])
#                 dp[i][1] = max(dp[-1][1]+0,dp[-1][0]-prices[i])
                dp[i][0] = max(0+0,float('-inf')+prices[i])
                dp[i][1] = max(float('-inf')+0,0-prices[i])
                continue
            dp[i][0] = max(dp[i-1][0]+0,dp[i-1][1]+prices[i])
            #根据题目要求，若当前有股票，必定是前天买入i-2->i
            dp[i][1] = max(dp[i-1][1]+0,dp[i-2][0]-prices[i])
                
        return dp[n-1][0]
    
    #交易次数没有限制，但是每次交易有手续费
    def solve_inf_fee(self,prices,fee):
        n = len(prices)
        dp = [[0,0] for i in range(n)]
        for i in range(n):
            if i == 0:
                #第-1天最大可以交易次数为0
#                 dp[i][0] = max(dp[-1][0]+0,dp[-1][1]+prices[i])
#                 dp[i][1] = max(dp[-1][1]+0,dp[-1][0]-prices[i])
                dp[i][0] = max(0+0,float('-inf')+prices[i])
                dp[i][1] = max(float('-inf')+0,0-prices[i]-2)
                continue
            dp[i][0] = max(dp[i-1][0]+0,dp[i-1][1]+prices[i])
            dp[i][1] = max(dp[i-1][1]+0,dp[i-1][0]-prices[i]-fee)
                
        return dp[n-1][0]
                

s = Solution()
prices = [1, 3, 2, 8, 4, 9]
s.solve_inf_fee(prices,2)

8

### 零钱兑换 leetcode 322  
- 题目
给定不同面额的硬币 coins 和一个总金额 amount。  
编写一个函数来计算可以凑成总金额所需的最少的硬币个数。如果没有任何一种硬币组合能组成总金额，返回 -1。  
示例 1:  
输入: coins = [1, 2, 5], amount = 11  
输出: 3  
解释: 11 = 5 + 5 + 1  
示例 2:  
输入: coins = [2], amount = 3  
输出: -1  
说明:  
你可以认为每种硬币的数量是无限的。  
- 解题目标
- 解题思路

In [25]:
#问题：硬币个数最少
#状态：amount
#选择；amount-[1,2,5]
#dp数组或递归函数：dp(amount)
#备忘录：dict
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        memo = dict()
        def solve(coins,amount):
            if amount in memo:
                return memo[amount]
            if amount == 0:
                return 0
            if amount < 0:
                return float('inf')
            min_res = float('inf')
            for coin in coins:
                min_res = min(solve(coins,amount-coin)+1,min_res)
                memo[amount] = min_res
            return min_res
        res = solve(coins,amount)
        if res == float('inf'):return -1
        else:return res
        
solution = Solution()
solution.coinChange([2],3)
        
    

-1

编辑距离 leetcode 72
给定两个单词 word1 和 word2，计算出将 word1 转换成 word2 所使用的最少操作数 。

你可以对一个单词进行如下三种操作：

插入一个字符
删除一个字符
替换一个字符
示例 1:

输入: word1 = "horse", word2 = "ros"
输出: 3
解释: 
horse -> rorse (将 'h' 替换为 'r')
rorse -> rose (删除 'r')
rose -> ros (删除 'e')
示例 2:

输入: word1 = "intention", word2 = "execution"
输出: 5
解释: 
intention -> inention (删除 't')
inention -> enention (将 'i' 替换为 'e')
enention -> exention (将 'n' 替换为 'x')
exention -> exection (将 'n' 替换为 'c')
exection -> execution (插入 'u')


In [36]:
class Solution:
    #将word1转换成word2
    def minDistance(self, word1: str, word2: str) -> int:
        #利用字典去除重复子问题
        res = dict()
        def dp(p1,p2):
            if (p1,p2) in res:
                return res[(p1,p2)]
            #p1到达边界,返回p2+1,即将p2及之前的字符插入到p1
            if p1 == -1:
                return p2+1
            #p2到达边界,返回p1+1,即将p1及之前的字符删除
            if p2 == -1:
                return p1+1
            #word1[p1]==word1[p1]，跳过
            if word1[p1] == word2[p2]:
                res[(p1,p2)] = dp(p1-1,p2-1)
            #word1[p1]！=word1[p1]，选择 插入；删除；替换 中最小的
            else:
                res[(p1,p2)] = min(dp(p1,p2-1),dp(p1-1,p2),dp(p1-1,p2-1))+1
            
            return res[(p1,p2)]
        
        return dp(len(word1)-1,len(word2)-1)

word1 = 'sea'
word2 = 'ate'
solution = Solution()
solution.minDistance(word1,word2)

3

鸡蛋掉落 leetcode 887
你将获得 K 个鸡蛋，并可以使用一栋从 1 到 N  共有 N 层楼的建筑。
每个蛋的功能都是一样的，如果一个蛋碎了，你就不能再把它掉下去。
你知道存在楼层 F ，满足 0 <= F <= N 任何从高于 F 的楼层落下的鸡蛋都会碎，从 F 楼层或比它低的楼层落下的鸡蛋都不会破。
每次移动，你可以取一个鸡蛋（如果你有完整的鸡蛋）并把它从任一楼层 X 扔下（满足 1 <= X <= N）。
你的目标是确切地知道 F 的值是多少。
无论 F 的初始值如何，你确定 F 的值的最小移动次数是多少？

示例 1：
输入：K = 1, N = 2
输出：2
解释：
鸡蛋从 1 楼掉落。如果它碎了，我们肯定知道 F = 0 。
否则，鸡蛋从 2 楼掉落。如果它碎了，我们肯定知道 F = 1 。
如果它没碎，那么我们肯定知道 F = 2 。
因此，在最坏的情况下我们需要移动 2 次以确定 F 是多少。

示例 2：
输入：K = 2, N = 6
输出：3

示例 3：
输入：K = 3, N = 14
输出：4

提示：
1 <= K <= 100
1 <= N <= 10000

题目要求什么：
题目-F 的值的最小移动次数
解释-在最坏的情况下我们需要移动 2 次以确定 F 是多少
最坏情况下F的最少移动次数
穷举所有最坏情况下的扔鸡蛋次数，选择最小
什么是最坏：鸡蛋碎在搜索区间穷尽的时候
最坏怎么计算：鸡蛋碎或者不碎的时候，产生的移动次数，取最大

状态：
有哪些状态（K，N）；状态如果转移（碎-K-1；，不碎）
选择：可以怎么选
dp数组或递归函数
备忘录

In [155]:
class Solution:
    def superEggDrop(self, K: int, N: int) -> int:
        memo = dict()
        #利用二分查找
        def solve2(num_egg,num_floor):
            if num_egg == 1:return num_floor
            if num_floor == 0:return 0
            if (num_egg,num_floor) in memo:
                return memo[(num_egg,num_floor)]
            res = float('inf')
            left,right = 1,num_floor
            while left <= right:
                mid = (left + right) // 2
                broken = solve2(num_egg-1,mid-1)
                not_broken = solve2(num_egg,num_floor-mid)
                if broken > not_broken:
                    right = mid - 1
                    res = min(res,broken+1)
                else:
                    left = mid + 1
                    res = min(res,not_broken+1)
            memo[(num_egg,num_floor)] = res
            return res
                
        
        def solve(num_egg,num_floor):
            #递归边界：只有一个鸡蛋时，只能一层一层往上找N，仍N次；没有鸡蛋时，返回0，因为没鸡蛋扔了
            if num_egg == 1:return num_floor
            if num_floor == 0:return num_floor
            if (num_egg,num_floor) in memo:
                return memo[(num_egg,num_floor)]
            #递归方程
            #穷举所有选择，每种选择下，每种状态产生的次数，取最大，然后，取每种选择的最小
            res = float('inf')
            for i in range(1,num_floor+1):
                #每种状态（碎，不碎）产生的次数，取最大；扔一次加1
                max_val = max(solve(num_egg-1,i-1),solve(num_egg,num_floor-i))+1
                #每种选择下，取每种选择的最小
                res = min(max_val,res)
            memo[(num_egg,num_floor)] = res
            return res
        
        return solve2(K,N)
    
solution = Solution()
solution.superEggDrop(4,2000)

16

数组的每个索引做为一个阶梯，第 i个阶梯对应着一个非负数的体力花费值 cost[i](索引从0开始)。
每当你爬上一个阶梯你都要花费对应的体力花费值，然后你可以选择继续爬一个阶梯或者爬两个阶梯。
您需要找到达到楼层顶部的最低花费。在开始时，你可以选择从索引为 0 或 1 的元素作为初始阶梯。

示例 1:
输入: cost = [10, 15, 20]
输出: 15
解释: 最低花费是从cost[1]开始，然后走两步即可到阶梯顶，一共花费15。

示例 2:
输入: cost = [1, 100, 1, 1, 1, 100, 1, 1, 100, 1]
输出: 6
解释: 最低花费方式是从cost[0]开始，逐个经过那些1，跳过cost[3]，一共花费6。

注意：
cost的长度将会在 [2, 1000]。
每一个 cost[i] 将会是一个Integer类型，范围为 [0, 999]

定义dp函数或数组
    一般是题目的直接目的
    当前的最小花费
    
定义状态
    可以变化的量
    走一步还是两步
    
定义选择
    每种状态的变化，可以做出的选择
    min(前一步的花费，前两步的花费)+当前的花费
    
min(solve(),solve())+

定义base case
    p=-1 v=0


In [13]:
class Solution:
    def minCostClimbingStairs(self, cost: List[int]) -> int:
        memo = dict()
        def solve(cost,p):
            if p >= len(cost):return 0
            if p in memo:
                return memo[p]
            memo[p] = min(solve(cost,p+1),solve(cost,p+2))+cost[p]
            return memo[p]
            
        return min(solve(cost,0),solve(cost,1))

cost = [10,15,20]
solution = Solution()
solution.minCostClimbingStairs(cost)
        

15

亚历克斯和李用几堆石子在做游戏。偶数堆石子排成一行，每堆都有正整数颗石子 piles[i] 。
游戏以谁手中的石子最多来决出胜负。石子的总数是奇数，所以没有平局。
亚历克斯和李轮流进行，亚历克斯先开始。 每回合，玩家从行的开始或结束处取走整堆石头。 这种情况一直持续到没有更多的石子堆为止，此时手中石子最多的玩家获胜。
假设亚历克斯和李都发挥出最佳水平，当亚历克斯赢得比赛时返回 true ，当李赢得比赛时返回 false 。

示例：

输入：[5,3,4,5]
输出：true
解释：
亚历克斯先开始，只能拿前 5 颗或后 5 颗石子 。
假设他取了前 5 颗，这一行就变成了 [3,4,5] 。
如果李拿走前 3 颗，那么剩下的是 [4,5]，亚历克斯拿走后 5 颗赢得 10 分。
如果李拿走后 5 颗，那么剩下的是 [3,4]，亚历克斯拿走后 4 颗赢得 9 分。
这表明，取前 5 颗石子对亚历克斯来说是一个胜利的举动，所以我们返回 true 。

提示：
2 <= piles.length <= 500
piles.length 是偶数。
1 <= piles[i] <= 500
sum(piles) 是奇数。


定义dp数组
dp[i][j] piles[i->j]中，两人的最佳挑选数量汇总
dp[i][j][0] 为alex的；p[i][j][1] 为lee的

定义状态
石头堆数

定义选择
选左边，还是右边

定义状态转移方程
alex dp[i][j][0] = max(piles[i]+dp[i+1][j][1],piles[j]+dp[i][j-1][1])

结果：
dp[i][j][0] - dp[i][j][1]


In [24]:
from typing import List
def print_list(lst):
    for i in lst:
        print(i)

In [31]:
class Solution:
    def stoneGame(self, piles: List[int]) -> bool:
        
        def solve(piles):
            piles_len = len(piles)
            #res[i][j] piles[i->j]中，两人的最佳挑选数量汇总
            res = [[[0,0] for i in range(piles_len)] for j in range(piles_len)]
            #base case 只有剩下一个石头堆的时候
            for i in range(piles_len):
                res[i][i][0] = piles[i]
                res[i][i][1] = 0
            #状态转移，迭代最优
            for gap in range(2,piles_len+1):
                for i in range(piles_len-gap+1):
                    j = i+gap-1
                    #alex选左边=当前选的，加下一轮选择的，下轮选择前lee先选，alex为后手
                    left = piles[i]+res[i+1][j][1]
                    #alex选右边
                    right = piles[j]+res[i][j-1][1]
                    #左边的大，则alex肯定选左边
                    if left > right:
                        res[i][j][0] = left
                        res[i][j][1] = res[i+1][j][0]
                    else:
                        res[i][j][0] = right
                        res[i][j][1] = res[i][j-1][0]
            return True if res[0][piles_len-1][0] - res[0][piles_len-1][1] > 0 else False
        
        return solve(piles)

piles = [5,3,4,5]
solution = Solution()
solution.stoneGame(piles)

True

给定一个区间的集合，找到需要移除区间的最小数量，使剩余区间互不重叠。
注意:
可以认为区间的终点总是大于它的起点。
区间 [1,2] 和 [2,3] 的边界相互“接触”，但没有相互重叠。
示例 1:

输入: [ [1,2], [2,3], [3,4], [1,3] ]
输出: 1
解释: 移除 [1,3] 后，剩下的区间没有重叠。

示例 2:
输入: [ [1,2], [1,2], [1,2] ]
输出: 2
解释: 你需要移除两个 [1,2] 来使剩下的区间没有重叠。

示例 3:
输入: [ [1,2], [2,3] ]
输出: 0
解释: 你不需要移除任何区间，因为它们已经是无重叠的了。


In [53]:
class Solution:
    def eraseOverlapIntervals(self, intervals: List[List[int]]) -> int:
        def solve(intervals):
            len_intervals = len(intervals)
            count = 0
            #按end从小到大排序
            intervals.sort(key=lambda x:x[1])
            #end往后找
            temp_stt = intervals[0][0]
            temp_end = intervals[0][1]
            for i in range(1,len_intervals):
                #相同的肯定要移除
                if intervals[i][0] == temp_stt and intervals[i][1] == temp_end:
                    count += 1
                #如果下个区间的stt小于当前的temp_end，要移除
                elif intervals[i][0] < temp_end:
                    count += 1
                else:
                    temp_stt = intervals[i][0]
                    temp_end = intervals[i][1]
            return count
        return solve(intervals)

intervals = [ [1,2],[1,2], [1,4], [1,3] ]
solution = Solution()
solution.eraseOverlapIntervals(intervals)

3

在二维空间中有许多球形的气球。
对于每个气球，提供的输入是水平方向上，气球直径的开始和结束坐标。由于它是水平的，所以y坐标并不重要，因此只要知道开始和结束的x坐标就足够了。开始坐标总是小于结束坐标。平面内最多存在104个气球。

一支弓箭可以沿着x轴从不同点完全垂直地射出。在坐标x处射出一支箭，若有一个气球的直径的开始和结束坐标为 xstart，xend， 且满足  xstart ≤ x ≤ xend，则该气球会被引爆。可以射出的弓箭的数量没有限制。 弓箭一旦被射出之后，可以无限地前进。我们想找到使得所有气球全部被引爆，所需的弓箭的最小数量。

Example:

输入:
[[10,16], [2,8], [1,6], [7,12]]

输出:
2

解释:
对于该样例，我们可以在x = 6（射爆[2,8],[1,6]两个气球）和 x = 11（射爆另外两个气球）。


In [79]:
class Solution:
    def findMinArrowShots(self, points: List[List[int]]) -> int:
        def solve(points):
            #按end从小到大排
            #初始要射n箭，存在一个重叠的区间，减一
            #重叠的区间中，取最大
            len_points = len(points)
            count = len_points
            if len_points <= 1:
                return len_points
            points.sort(key=lambda x:(x[1],x[0]))
            temp_stt = points[0][0]
            temp_end = points[0][1]
            for i in range(1,len_points):
                #位置相同，重叠
                #stt小于temp_end，重叠，如果end大于temp_end，则temp_end=end
                if temp_stt == points[i][0] and temp_end == points[i][1]:
                    count -= 1
                elif points[i][0] <= temp_end:
                    count -= 1
                    temp_end = min(temp_end,points[i][1])
                    temp_stt = max(temp_stt,points[i][1])
                else:
                    temp_stt = points[i][0]
                    temp_end = points[i][1]
            return count
        
        return solve(points)
    
points = [[3,9],[7,12],[3,8],[6,8],[9,10],[2,9],[0,9],[3,9],[0,6],[2,8]]#[[1,2],[2,3],[3,4],[4,5]]#[[10,16], [2,8], [1,6], [7,12]]
solution = Solution()
solution.findMinArrowShots(points)

2

假设你正在爬楼梯。需要 n 阶你才能到达楼顶。
每次你可以爬 1 或 2 个台阶。你有多少种不同的方法可以爬到楼顶呢？
注意：给定 n 是一个正整数。

示例 1：
输入： 2
输出： 2
解释： 有两种方法可以爬到楼顶。
1.  1 阶 + 1 阶
2.  2 阶

示例 2：
输入： 3
输出： 3
解释： 有三种方法可以爬到楼顶。
1.  1 阶 + 1 阶 + 1 阶
2.  1 阶 + 2 阶
3.  2 阶 + 1 阶


In [20]:
class Solution:
    def climbStairs(self, n: int) -> int:
        def solve(n):
            if n <= 2:return n
            pre = 2
            prepre = 1
            index = 3
            res = 0
            while index <= n:
                res = prepre + pre
                pre,prepre = res,pre
                index += 1
            return res
        return solve(n)

solution = Solution()
solution.climbStairs(2)

2

你是一个专业的小偷，计划偷窃沿街的房屋。每间房内都藏有一定的现金，影响你偷窃的唯一制约因素就是
相邻的房屋装有相互连通的防盗系统，如果两间相邻的房屋在同一晚上被小偷闯入，系统会自动报警。

给定一个代表每个房屋存放金额的非负整数数组，计算你在不触动警报装置的情况下，能够偷窃到的最高金额。

示例 1:

输入: [1,2,3,1]
输出: 4
解释: 偷窃 1 号房屋 (金额 = 1) ，然后偷窃 3 号房屋 (金额 = 3)。
     偷窃到的最高金额 = 1 + 3 = 4 。
     
示例 2:
输入: [2,7,9,3,1]
输出: 12
解释: 偷窃 1 号房屋 (金额 = 2), 偷窃 3 号房屋 (金额 = 9)，接着偷窃 5 号房屋 (金额 = 1)。
     偷窃到的最高金额 = 2 + 9 + 1 = 12 。

状态：偷窃金额，剩余房屋数
选择：偷，不偷
状态转移方程：
base case:

第一或第二个，肯定会偷
下个或下下个，肯定回偷
max(solve(i+2,nums),solve(i+3,nums))
max(solve(0,nums),solve(1,nums))
dp[i] 第i家时打劫的金额
dp[i] = max(dp[i+1],dp[i+2]+nums[i]) : max(不打劫，打劫)

In [14]:
class Solution:
    def rob(self, nums: List[int]) -> int:
        def solve(nums):
            res = [0 for i in range(len(nums)+2)]
            for i in range(len(nums)-1,-1,-1):
                res[i] = max(res[i+1],res[i+2]+nums[i])
            return res[0]
        return solve(nums)
cost = [2,7,9,3,1]
solution = Solution()
solution.rob(cost)

12

你是一个专业的小偷，计划偷窃沿街的房屋，每间房内都藏有一定的现金。这个地方所有的房屋都围成一圈，这意味着第一个房屋和最后一个房屋是紧挨着的。同时，相邻的房屋装有相互连通的防盗系统，如果两间相邻的房屋在同一晚上被小偷闯入，系统会自动报警。

给定一个代表每个房屋存放金额的非负整数数组，计算你在不触动警报装置的情况下，能够偷窃到的最高金额。

示例 1:

输入: [2,3,2]
输出: 3
解释: 你不能先偷窃 1 号房屋（金额 = 2），然后偷窃 3 号房屋（金额 = 2）, 因为他们是相邻的。
示例 2:

输入: [1,2,3,1]
输出: 4
解释: 你可以先偷窃 1 号房屋（金额 = 1），然后偷窃 3 号房屋（金额 = 3）。
     偷窃到的最高金额 = 1 + 3 = 4 。

假设，从第一个开始偷，则最大只能偷到倒数第二个
假设，从第二个开始偷，则最大只能偷到倒数第一个
max(solve(0,nums,flag=1),solve(1,nums,flag=2))
max(solve(i+2,nums),solve(i+3,nums))

In [70]:
class Solution:
    def rob(self, nums: List[int]) -> int:
        if not nums:return 0
        if len(nums) <= 3:return max(nums)
        memo = dict()
        def solve(curr_idx,nums,flag):
            if flag == '1' and curr_idx == len(nums)-2:return nums[curr_idx]
            if flag == '1' and curr_idx >= len(nums)-1:return 0
            if flag == '2' and curr_idx == len(nums)-1:return nums[curr_idx]
            if flag == '2' and curr_idx >= len(nums):return 0
            res = float('-inf')
            for i in range(curr_idx+2,len(nums)):
                res = max(solve(i,nums,flag),res)
            res += nums[curr_idx]
            return res
        return max(solve(0,nums,flag='1'),solve(1,nums,flag='2'))
cost = [1, 2, 3, 4, 5, 1, 2, 3, 4, 5]#13524#24135#13535
solution = Solution()
solution.rob(cost)

15

In [51]:
'1' +'-'+ str(1)

'1-1'

给定字符串 s 和 t ，判断 s 是否为 t 的子序列。

你可以认为 s 和 t 中仅包含英文小写字母。字符串 t 可能会很长（长度 ~= 500,000），而 s 是个短字符串（长度 <=100）。

字符串的一个子序列是原始字符串删除一些（也可以不删除）字符而不改变剩余字符相对位置形成的新字符串。（例如，"ace"是"abcde"的一个子序列，而"aec"不是）。

示例 1:
s = "abc", t = "ahbgdc"

返回 true.

示例 2:
s = "axc", t = "ahbgdc"

返回 false.


In [35]:
#s 第一个字符，遍历t 是否有，有则记录第一个位置 t = [i+:]
#循环
class Solution:
    def __init__(self):
        self.count = 0
    def isSubsequence(self, s: str, t: str) -> bool:
        def solve(s,t):
            if not s or not t:return
            stt = s[0]
            for i in range(len(t)):
                if t[i] == stt:
                    self.count += 1
                    solve(s[1:],t[i+1:])
                    break
        solve(s,t)
        return len(s) == self.count
    
s = 'leeeeetcode'
t = 'yyyyylyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyeyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyeyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyeyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyytyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyycyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyoyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyydyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyeyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy'
solution = Solution()
solution.isSubsequence(s,t)

False

In [4]:
a = [1]
a[1:]

[]